In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
import os
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
from sklearn.metrics import r2_score
from scipy.interpolate import interp1d
import tensorflow_datasets as tfds
from collections import Counter
import scipy.ndimage
from tensorflow.keras.callbacks import ModelCheckpoint
import keras
import gc
import random
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import tarfile
import urllib.request
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torch.nn.functional as F
from scipy.stats import gaussian_kde
from numba import jit
from torchvision.datasets import Places365
from tqdm import tqdm  # 導入進度條庫
import json
import torch.nn as nn

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
torch.backends.cudnn.benchmark = True
limit = 1000

# 設定 Dropout 機率
dropout_rate = 0.1
num_samples = 128

# bzq modifying
len_x_target = 20
len_y_target = 20
stride_x_target = 10
stride_y_target = 10

# mean, std proportion
alpha = 0.5

bins_size = 30  # 統計採樣數
poly_degree = bins_size - 1
window_size = 1

#target image preprocessing
angle = 0
pixels = 0

In [ ]:
# 設置設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),  # Convert to tensor (values in range 0–1)
    transforms.Lambda(lambda x: x * 255.0),  # Scale to range 0–255
    transforms.Lambda(lambda x: x[[2, 1, 0], ...]),  # Convert RGB to BGR
    transforms.Normalize(mean=[103.939, 116.779, 123.68], std=[1.0, 1.0, 1.0])  # Subtract mean
])

# 建立隨機影像數據集（不依賴 Places365）
class RandomImageDataset(torch.utils.data.Dataset):
    def __init__(self, num_images=1000, image_size=(224, 224)):
        self.num_images = num_images
        self.image_size = image_size

    def __len__(self):
        return self.num_images

    def __getitem__(self, idx):
        random_image = np.random.randint(0, 256, (self.image_size[1], self.image_size[0], 3), dtype=np.uint8)
        image = Image.fromarray(random_image)
        return transform(image)

# 使用自定義隨機影像數據集
dataset = RandomImageDataset(num_images=limit)

In [ ]:
model_bzq = models.vgg16(weights=False)

# 修改 classifier 最後一層，適應 365 類別
num_features = model_bzq.classifier[6].in_features
model_bzq.classifier[6] = nn.Linear(num_features, 365)

# 加載保存的權重
if os.path.exists("vgg16_places365.pt"):
    checkpoint = torch.load("vgg16_places365.pt")
    
    # 映射權重 (Caffe -> PyTorch)
    mapped_state_dict_bzq = {
        # Conv1 Layers
        "features.0.weight": checkpoint["conv1_1.weight"],
        "features.0.bias": checkpoint["conv1_1.bias"],
        "features.2.weight": checkpoint["conv1_2.weight"],
        "features.2.bias": checkpoint["conv1_2.bias"],
        
        # Conv2 Layers
        "features.5.weight": checkpoint["conv2_1.weight"],
        "features.5.bias": checkpoint["conv2_1.bias"],
        "features.7.weight": checkpoint["conv2_2.weight"],
        "features.7.bias": checkpoint["conv2_2.bias"],
        
        # Conv3 Layers
        "features.10.weight": checkpoint["conv3_1.weight"],
        "features.10.bias": checkpoint["conv3_1.bias"],
        "features.12.weight": checkpoint["conv3_2.weight"],
        "features.12.bias": checkpoint["conv3_2.bias"],
        "features.14.weight": checkpoint["conv3_3.weight"],
        "features.14.bias": checkpoint["conv3_3.bias"],
        
        # Conv4 Layers
        "features.17.weight": checkpoint["conv4_1.weight"],
        "features.17.bias": checkpoint["conv4_1.bias"],
        "features.19.weight": checkpoint["conv4_2.weight"],
        "features.19.bias": checkpoint["conv4_2.bias"],
        "features.21.weight": checkpoint["conv4_3.weight"],
        "features.21.bias": checkpoint["conv4_3.bias"],
        
        # Conv5 Layers
        "features.24.weight": checkpoint["conv5_1.weight"],
        "features.24.bias": checkpoint["conv5_1.bias"],
        "features.26.weight": checkpoint["conv5_2.weight"],
        "features.26.bias": checkpoint["conv5_2.bias"],
        "features.28.weight": checkpoint["conv5_3.weight"],
        "features.28.bias": checkpoint["conv5_3.bias"],
        
        # Fully Connected Layers
        "classifier.0.weight": checkpoint["fc6.weight"],
        "classifier.0.bias": checkpoint["fc6.bias"],
        "classifier.3.weight": checkpoint["fc7.weight"],
        "classifier.3.bias": checkpoint["fc7.bias"],
        "classifier.6.weight": checkpoint["fc8a.weight"],
        "classifier.6.bias": checkpoint["fc8a.bias"],
    }
    #print(model.features)

    model_bzq.load_state_dict(mapped_state_dict_bzq, strict=False)



# 初始化 VGG16，無預訓練權重
model = models.vgg16(weights=False)

# 修改 classifier 最後一層，適應 365 類別
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 365)

 

# 為 features 添加 Dropout
new_features = []
for layer in model.features:
    new_features.append(layer)
    if isinstance(layer, nn.ReLU):  # 在 ReLU 之後插入 Dropout
        new_features.append(nn.Dropout(p=dropout_rate))

model.features = nn.Sequential(*new_features)

# 定義新的 classifier，刪除第3個和第7個 Dropout
new_classifier = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.1, inplace=False),  # 保留 p=0.1
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.1, inplace=False),  # 保留 p=0.1
    nn.Linear(4096, 365)
)

# 替換原本的 classifier
model.classifier = new_classifier

# 加載保存的權重
if os.path.exists("vgg16_places365.pt"):
    checkpoint = torch.load("vgg16_places365.pt")
    
    # 映射權重 (Caffe -> PyTorch)
    mapped_state_dict = {  
        "features.0.weight": checkpoint["conv1_1.weight"],
        "features.0.bias": checkpoint["conv1_1.bias"],
        "features.3.weight": checkpoint["conv1_2.weight"],
        "features.3.bias": checkpoint["conv1_2.bias"],
        "features.7.weight": checkpoint["conv2_1.weight"],
        "features.7.bias": checkpoint["conv2_1.bias"],
        "features.10.weight": checkpoint["conv2_2.weight"],
        "features.10.bias": checkpoint["conv2_2.bias"],
        "features.14.weight": checkpoint["conv3_1.weight"],
        "features.14.bias": checkpoint["conv3_1.bias"],
        "features.17.weight": checkpoint["conv3_2.weight"],
        "features.17.bias": checkpoint["conv3_2.bias"],
        "features.20.weight": checkpoint["conv3_3.weight"],
        "features.20.bias": checkpoint["conv3_3.bias"],
        "features.24.weight": checkpoint["conv4_1.weight"],
        "features.24.bias": checkpoint["conv4_1.bias"],
        "features.27.weight": checkpoint["conv4_2.weight"],
        "features.27.bias": checkpoint["conv4_2.bias"],
        "features.30.weight": checkpoint["conv4_3.weight"],
        "features.30.bias": checkpoint["conv4_3.bias"],
        "features.34.weight": checkpoint["conv5_1.weight"],
        "features.34.bias": checkpoint["conv5_1.bias"],
        "features.37.weight": checkpoint["conv5_2.weight"],
        "features.37.bias": checkpoint["conv5_2.bias"],
        "features.40.weight": checkpoint["conv5_3.weight"],
        "features.40.bias": checkpoint["conv5_3.bias"],
        "classifier.0.weight": checkpoint["fc6.weight"],
        "classifier.0.bias": checkpoint["fc6.bias"],
        "classifier.3.weight": checkpoint["fc7.weight"],
        "classifier.3.bias": checkpoint["fc7.bias"],
        "classifier.6.weight": checkpoint["fc8a.weight"],
        "classifier.6.bias": checkpoint["fc8a.bias"],
    }
    #print(model.features)

    model.load_state_dict(mapped_state_dict, strict=False)

# **啟用 MC Dropout**
def enable_dropout(m):
    if isinstance(m, nn.Dropout):
        m.train()


model = model.to(device)
model_bzq = model_bzq.to(device)
if dropout_rate > 0.0:
    print("enable dropout!")
    model.apply(enable_dropout)  # 讓 Dropout 在推論時啟用
model.eval()  # 設定評估模式，但 Dropout 仍作用
model_bzq.eval()


In [ ]:
# 得到了bzq 正確的函數，拿來做imagenet 正確的預測

def single_data_bzq_mask_preprocessing_imagenet(original_data, start_x, start_y, len_x, len_y, magnification):
    if len_x <= 0 or len_y <= 0:
        return original_data
    new_data = np.copy(original_data)
    new_data[:, start_y:start_y + len_y, start_x:start_x + len_x] *= magnification
    #new_data[start_y:start_y + len_y, start_x:start_x + len_x, :] *= magnification
    return new_data


#print(random_num_for_bzq_mask_imagenet)

def single_data_bzq_mask_preprocessing_imagenet_random_global(original_data, start_x, start_y, len_x, len_y, random_num_for_bzq_mask_imagenet):
    if len_x <= 0 or len_y <= 0:
        return original_data
    new_data = np.copy(original_data.cpu().numpy())
    new_data = new_data.squeeze(0)
    random_num_for_bzq_mask_imagenet = random_num_for_bzq_mask_imagenet[:, :len_y, :len_x] 
    new_data[:, start_y:start_y + len_y, start_x:start_x + len_x] = random_num_for_bzq_mask_imagenet
    return new_data


bzq = []
correct_predictions_imagenet = []
incorrect_predictions_imagenet = []
bzq_imagenet = []

nll_bzq = []

#bzq = 0的時候，提取mask之下的softmax
correct_predictions_bzq_zero_softmax_mean = []
correct_predictions_bzq_zero_softmax_std = []
incorrect_predictions_bzq_zero_softmax_mean = []
incorrect_predictions_bzq_zero_softmax_std = []


In [ ]:
##################################################################################################################################
def reverse_normalize(image, mean, std):
    # 確保均值和標準差是 numpy array，以支持逐像素計算
    mean = np.array(mean).reshape(1, 1, -1)  # [1, 1, C]
    std = np.array(std).reshape(1, 1, -1)    # [1, 1, C]

    # 反標準化公式
    image = (image * std) + mean
    return np.clip(image, 0, 255)  # 確保範圍在 [0, 255]

def normalize(image, mean, std):
    # 將影像標準化
    mean = np.array(mean).reshape(1, 1, -1)  # [1, 1, C]
    std = np.array(std).reshape(1, 1, -1)    # [1, 1, C]
    image = (image - mean) / std
    return image

# Custom dataset to apply elastic_transform
# 自定義數據集
class IndexBasedTransformDataset(Dataset):
    def __init__(self, original_dataset, transforms_by_range, repeat_factor=1):
        """
        original_dataset: 原始 PyTorch Dataset
        transforms_by_range: 列表，每個元素是 (start_index, end_index, transform_function)
        repeat_factor: 數據集重複的倍數
        """
        self.original_dataset = original_dataset
        self.transforms_by_range = transforms_by_range
        self.repeat_factor = repeat_factor  # 數據集重複幾次

    def __len__(self):
        return len(self.original_dataset) * self.repeat_factor

    def __getitem__(self, idx):
        # 對數據索引進行取模，將重複部分對應回原始數據
        original_idx = idx % len(self.original_dataset)
        image, label = self.original_dataset[original_idx]
        
        # 轉換影像形狀為 [H, W, C]
        if isinstance(image, torch.Tensor):
            image = image.permute(1, 2, 0).numpy()  # 從 [C, H, W] 變為 [H, W, C]
        
        # Step 1: 反標準化
        image = reverse_normalize(image, mean=[103.939, 116.779, 123.68], std=[1.0, 1.0, 1.0])

        # Step 2: 影像增強
        for start, end, transform in self.transforms_by_range:
            if start <= idx < end:
                if transform is not None:
                    image = transform(image)
                break
        
        # Step 3: 標準化
        image = normalize(image, mean=[103.939, 116.779, 123.68], std=[1.0, 1.0, 1.0])

        # 還原影像形狀為 [C, H, W]
        image = np.transpose(image, (2, 0, 1))  # 從 [H, W, C] 變回 [C, H, W]
        return image, label
    
test_dataset = dataset
image = test_dataset[0]
print(image.shape)
# 創建 DataLoader
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=6, pin_memory=True)

In [ ]:
conf_vanilla = []
# 預測並顯示分數
print(test_dataset[0].shape)

correct = 0
total = 0


'''fig, axes = plt.subplots(3, 5, figsize=(15, 9))
class_names = {0:'brightness', 1:'contrast', 2:'defocus_blur', 3:'elastic_transform', 4:'fog', 
                    5:'gaussian_blur', 6:'gaussian_noise', 7:'glass_blur', 8:'impulse_noise', 
                    9:'pixelate', 10:'saturate', 11:'shot_noise', 12:'spatter', 13:'speckle_noise', 14:'zoom_blur'}  # 你的類別名稱
for i in range(15):
    index = limit * i  # 計算索引
    img, _ = test_dataset[index]  # 讀取影像

    # 反正規化影像
    img = reverse_normalize(img.transpose(1, 2, 0), mean=[103.939, 116.779, 123.68], std=[1.0, 1.0, 1.0]) / 255
    
    ax = axes[i // 5, i % 5]  # 計算子圖位置
    ax.imshow(img)
    ax.set_title(f"{class_names[i]}", fontsize=12)
    ax.axis("off")  # 隱藏座標軸

plt.tight_layout()
plt.show()'''

In [ ]:
# bzq modifying
len_x = len_x_target
len_y = len_y_target
stride_x = stride_x_target
stride_y = stride_y_target
batch_size = 1  # 設定批次大小

original_predictions_imagenet = []

with torch.no_grad():
    for batch_data in test_loader:
        batch_data = batch_data.to(device).float()
        batch_predictions_imagenet = model(batch_data).cpu().numpy()
        original_predictions_imagenet.append(batch_predictions_imagenet)

original_predictions_imagenet = np.vstack(original_predictions_imagenet)

# 使用正確的方式來訪問 test_dataset 中的標籤
test_labels = [test_dataset[i][1] for i in range(len(test_dataset))]

predicted_labels = np.argmax(original_predictions_imagenet, axis=1)



In [ ]:
import torch
import numpy as np
from collections import Counter
import torch.nn.functional as F

random_num_for_bzq_mask_imagenet = np.random.randint(0, 256, (3, len_y_target, len_x_target)).astype(np.float32) / 255.0

def process_batch(batch_data, len_y, stride_y, len_x, stride_x, device, model, model_bzq, alpha, bzq, mc_samples):
    if len(batch_data) == 0:
        return bzq, np.array([])

    targets = []

    for i in range(0, 224 - len_y, stride_y):
        for j in range(0, 224 - len_x, stride_x):
            #target = single_data_bzq_mask_preprocessing_imagenet(batch_data, i, j, len_x, len_y, 0)
            target = single_data_bzq_mask_preprocessing_imagenet_random_global(batch_data, i, j, len_x, len_y, random_num_for_bzq_mask_imagenet)
            targets.append(target)
    
    targets_tensor = torch.from_numpy(np.vstack(targets).reshape(-1, 3, 224, 224)).float().to(device)
    predictions = model_bzq(targets_tensor)
    
    #temp, _ = torch.max(F.softmax(predictions), dim=1)
    #temp = temp.cpu().numpy()
    
    max_bzq_indices = torch.argmax(predictions, dim=1).cpu().numpy()
    
    # 計算 softmax
    softmax_predictions = F.softmax(predictions, dim=1)

    counter = Counter(max_bzq_indices)
    most_common_num, most_common_count = counter.most_common(1)[0]
    
    temp = softmax_predictions[:, most_common_num].cpu().numpy()
    
    mc_predictions = []
    
    for _ in range(mc_samples):  # **進行多次 forward**
        predictions = model(batch_data.float())
        mc_predictions.append(predictions.unsqueeze(0))  # **儲存 MC Dropout 結果**

    # **堆疊所有 MC Dropout 結果**
    mc_predictions = torch.cat(mc_predictions, dim=0)  # (mc_samples, batch_size, num_classes)

    # **計算 MC Dropout 之後的平均**
    mean_predictions = mc_predictions.mean(dim=0)  # 均值 (batch_size, num_classes)

    # **計算 softmax**
    softmax_predictions_dropout = F.softmax(mean_predictions, dim=1)
    
    temp_dropout = softmax_predictions_dropout[:, most_common_num].cpu().numpy()
    
    bzq.append((alpha * np.mean(temp) + (1 - alpha) * (2.0 / np.pi * np.arctan(1.0 / np.std(temp)))))
    #print(softmax_predictions.shape)
    #print(softmax_predictions_dropout.shape)
    # 將兩個 softmax 結果相加除以 2
    mixed_soft_predictions = (softmax_predictions.mean(dim=0, keepdim=True) + softmax_predictions_dropout) / 2  # shape: (batch_size, num_classes)
    entropy = -torch.sum(mixed_soft_predictions * torch.log(mixed_soft_predictions + 1e-10), dim=1)  # shape: (batch_size,)
    return bzq, temp, temp_dropout, most_common_num, entropy.cpu().numpy()

score_dropout = []
index_count = 0
entropies = []
with torch.no_grad():
    for batch_data in test_loader:
        batch_data = batch_data.to(device)  # 將批次影像搬到指定裝置
        if len(batch_data) > 0:
            bzq, temp, temp_dropout, predicted_label, entropy = process_batch(
                batch_data, len_y, stride_y, len_x, stride_x, device, model, model_bzq, alpha, bzq, num_samples
            )

            score_dropout.append(temp_dropout)
            entropies.append(entropy)
            
            # 影像預處理
            original_data = single_data_bzq_mask_preprocessing_imagenet(batch_data, 0, 0, 0, 0, 0)
            original_prediction = model(torch.tensor(original_data.reshape(-1, 3, 224, 224)).float().to(device))
            max_original_index = torch.argmax(original_prediction).item()

            # 根據 `bzq` 值分類統計
            if bzq[-1] == 0.0:
                if (len(bzq) - 1) in correct_predictions_imagenet:
                    correct_predictions_bzq_zero_softmax_mean.append(np.mean(temp))
                    correct_predictions_bzq_zero_softmax_std.append(np.std(temp))
                else:
                    incorrect_predictions_bzq_zero_softmax_mean.append(np.mean(temp))
                    incorrect_predictions_bzq_zero_softmax_std.append(np.std(temp))

bzq_imagenet = np.array(bzq)
score_dropout = np.array(score_dropout)


In [ ]:
entropies = np.array(entropies).flatten()
#print(entropies)
# 計算熵的分佈（分桶）
bins = np.linspace(min(entropies), max(entropies), 15) 

hist, bin_edges = np.histogram(entropies, bins=bins)  # 計算直方圖數據
print(bin_edges[:-1], hist)
# 轉換為折線圖
plt.figure(figsize=(10, 5))
plt.plot(bin_edges[:-1], hist, linestyle='-', marker='o')

# 設定標題與軸標籤
plt.title("Entropy")
plt.xlabel("Entropy")
plt.ylabel("Frequency")
plt.legend()
plt.grid(True)
plt.show()